In [3]:
# upload webscraped data sets 

import pandas as pd
import string
import re
import numpy as np
import datetime as dt
from datetime import datetime


In [4]:
# read all data sets that were webscraped

all_season_results_df = pd.read_csv('all_season_results_final.csv')
all_boxscore_results_df = pd.read_csv('all boxscore results.csv')

In [5]:
all_season_results_df['game_number_team1']=all_season_results_df['game_number_team1'].astype(int)
all_season_results_df['game_number_team2']=all_season_results_df['game_number_team2'].astype(int)
all_season_results_df['home_flag_team2']=all_season_results_df['home_flag_team2'].astype(int)
all_season_results_df['season']=all_season_results_df['season'].astype(int)
all_season_results_df.head()

,date,home_flag_team1,home_flag_team2,score1,score2,season,team1,team2,win_flag_team1,win_flag_team2,game_number_team1,game_number_team2,b2b_flag_team1,b2b_flag_team2,travel_flag_team1,travel_flag_team2
0,1999-02-05,1,0,100,83,1999,ATLANTA HAWKS,CLEVELAND CAVALIERS,1,0,1,1,0,0,0,1
1,1999-02-06,1,0,111,106,1999,ATLANTA HAWKS,NEW JERSEY NETS,1,0,2,1,1,0,0,1
2,1999-02-07,0,1,69,79,1999,ATLANTA HAWKS,NEW JERSEY NETS,0,1,3,2,1,1,1,1
3,1999-02-09,0,1,87,71,1999,ATLANTA HAWKS,CHICAGO BULLS,1,0,4,3,0,0,1,1
4,1999-02-11,1,0,100,88,1999,ATLANTA HAWKS,CHARLOTTE HORNETS,1,0,5,5,0,1,1,1


In [6]:
# creating data set that only includes season games, and a unique key that represents the team, season, and game number
# join data sets based on team1_gameid and team2_gameid

all_season_results_df = all_season_results_df[all_season_results_df.game_number_team1 <= 82]
all_season_results_df['team1_game_id'] = all_season_results_df['team1'] + '-' + all_season_results_df['season'].apply(str) + '-' + all_season_results_df['game_number_team1'].apply(str)
all_season_results_df['team2_game_id'] = all_season_results_df['team2'] + '-' + all_season_results_df['season'].apply(str) + '-' + all_season_results_df['game_number_team2'].apply(str)
all_season_results_df.head()

,date,home_flag_team1,home_flag_team2,score1,score2,season,team1,team2,win_flag_team1,win_flag_team2,game_number_team1,game_number_team2,b2b_flag_team1,b2b_flag_team2,travel_flag_team1,travel_flag_team2,team1_game_id,team2_game_id
0,1999-02-05,1,0,100,83,1999,ATLANTA HAWKS,CLEVELAND CAVALIERS,1,0,1,1,0,0,0,1,ATLANTA HAWKS-1999-1,CLEVELAND CAVALIERS-1999-1
1,1999-02-06,1,0,111,106,1999,ATLANTA HAWKS,NEW JERSEY NETS,1,0,2,1,1,0,0,1,ATLANTA HAWKS-1999-2,NEW JERSEY NETS-1999-1
2,1999-02-07,0,1,69,79,1999,ATLANTA HAWKS,NEW JERSEY NETS,0,1,3,2,1,1,1,1,ATLANTA HAWKS-1999-3,NEW JERSEY NETS-1999-2
3,1999-02-09,0,1,87,71,1999,ATLANTA HAWKS,CHICAGO BULLS,1,0,4,3,0,0,1,1,ATLANTA HAWKS-1999-4,CHICAGO BULLS-1999-3
4,1999-02-11,1,0,100,88,1999,ATLANTA HAWKS,CHARLOTTE HORNETS,1,0,5,5,0,1,1,1,ATLANTA HAWKS-1999-5,CHARLOTTE HORNETS-1999-5


- Types of features:
        - For each team and it's opponent
        - Season Stats: % of total games completed, win pct, Offensive Rating, Defensive Rating, Pace        
        - Rolling 10 game stats:
        - Misc stats: current team home/away flag,
        - Team Stats: Home or Away,   
        - Team Record Stats: % of games played this season, Season Win Percentage, Rolling 10 game win %, Rolling 10 game
        Opponent win %, Win Percentage (based on home/away flag) 
        - Opponent Stats: Home or Away, Offensive Efficiency, Defensive Efficiency, Pace 
        - Opponent Record Stats: Season Win Percentage, Rolling 10 game win %, Rolling 10 game Opponent win %,
                                 Win Percentage (based on home/away flag)
        - Matchup Stats: Rolling 10 game win % against Current Opponent

In [7]:

all_boxscore_results_df = all_boxscore_results_df[all_boxscore_results_df.game_number <= 82]
all_boxscore_results_df['game_id'] = all_boxscore_results_df['team'] + '-' + all_boxscore_results_df['season'].apply(str) + '-' + all_boxscore_results_df['game_number'].apply(str)
print(len(all_boxscore_results_df))


# First, create all the advanced metrics for each game:
all_boxscore_results_df['posessions'] = all_boxscore_results_df['attempted_field_goals']+(.4*all_boxscore_results_df['attempted_free_throws'])-all_boxscore_results_df['offensive_rebounds']+all_boxscore_results_df['turnovers']
all_boxscore_results_df['team_ortg'] = all_boxscore_results_df['points']/all_boxscore_results_df['posessions']*100
all_boxscore_results_df['ast%'] = (all_boxscore_results_df['assists']/all_boxscore_results_df['made_field_goals'])
all_boxscore_results_df['true_shooting_attempts'] = all_boxscore_results_df['attempted_field_goals']+(.44*all_boxscore_results_df['attempted_free_throws'])
all_boxscore_results_df['ts%'] = all_boxscore_results_df['points']/(2*all_boxscore_results_df['true_shooting_attempts'])
all_boxscore_results_df['efg%'] = (all_boxscore_results_df['made_field_goals']+(.5*all_boxscore_results_df['made_three_point_field_goals']))/all_boxscore_results_df['attempted_field_goals']

advanced_stats_team = all_boxscore_results_df[['game_id','posessions','team_ortg','ast%','ts%','efg%',
                                               'offensive_rebounds','defensive_rebounds','blocks',
                                               'steals','personal_fouls','turnovers','attempted_field_goals',
                                                'attempted_free_throws','attempted_three_point_field_goals',
                                                'made_field_goals','made_free_throws','made_three_point_field_goals']]
advanced_stats_team1 = advanced_stats_team.rename(columns={"game_id":"game_number_team1",
                                                          "posessions":"poss_team1",
                                                          "team_ortg":"ortg_team1",
                                                          "ast%":"ast%_team1",
                                                          "ts%":"ts%_team1",
                                                          "efg%":"efg%_team1",
                                                          "offensive_rebounds":"oreb_team1",
                                                          "defensive_rebounds":"dreb_team1",
                                                          "blocks":"blk_team1",
                                                          "steals":"stl_team1",
                                                          "personal_fouls":"pf_team1",
                                                          "turnovers":"to_team1",
                                                           "attempted_field_goals": "fga_team1",
                                                           "attempted_free_throws": "fta_team1",
                                                           "attempted_three_point_field_goals":"3fga_team1",
                                                           "made_field_goals": "fgm_team1",
                                                           "made_free_throws": "ftm_team1",
                                                           "made_three_point_field_goals":"3fgm_team1"
                                                          })
advanced_stats_team2 = advanced_stats_team.rename(columns={"game_id":"game_number_team2",
                                                          "posessions":"poss_team2",
                                                          "team_ortg":"ortg_team2",
                                                          "ast%":"ast%_team2",
                                                          "ts%":"ts%_team2",
                                                          "efg%":"efg%_team2",
                                                          "offensive_rebounds":"oreb_team2",
                                                          "defensive_rebounds":"dreb_team2",
                                                          "blocks":"blk_team2",
                                                          "steals":"stl_team2",
                                                          "personal_fouls":"pf_team2",
                                                          "turnovers":"to_team2",
                                                           "attempted_field_goals": "fga_team2",
                                                           "attempted_free_throws": "fta_team2",
                                                           "attempted_three_point_field_goals": "3fga_team2",
                                                           "made_field_goals": "fgm_team2",
                                                           "made_free_throws": "ftm_team2",
                                                           "made_three_point_field_goals": "3fgm_team2"})

51987


In [8]:
advanced_stats_team2.head()

,game_number_team2,poss_team2,ortg_team2,ast%_team2,ts%_team2,efg%_team2,oreb_team2,dreb_team2,blk_team2,stl_team2,pf_team2,to_team2,fga_team2,fta_team2,3fga_team2,fgm_team2,ftm_team2,3fgm_team2
0,ATLANTA HAWKS-1999-1,91.2,109.649123,0.484848,0.556050,0.478873,10,28,8,8,25,13,71,43,10,33,32,2
1,ATLANTA HAWKS-1999-2,93.0,119.354839,0.500000,0.600649,0.477273,12,30,6,4,34,15,66,60,7,30,48,3
2,ATLANTA HAWKS-1999-3,91.4,75.492341,0.391304,0.385733,0.307692,16,20,5,11,31,19,78,26,11,23,21,2
3,ATLANTA HAWKS-1999-4,85.6,101.635514,0.484848,0.525616,0.492857,12,33,7,9,20,16,70,29,7,33,18,3
4,ATLANTA HAWKS-1999-5,89.8,111.358575,0.615385,0.537172,0.500000,12,38,4,5,15,10,79,32,5,39,21,1


In [9]:
# merge data to game results for opponent and current team

game_results = all_season_results_df.merge(advanced_stats_team1, how = 'inner',  left_on='team1_game_id', right_on='game_number_team1')
game_results = game_results.merge(advanced_stats_team2, how = 'inner',  left_on='team2_game_id', right_on='game_number_team2')
game_results.to_csv('game_results.csv')
game_results.columns

Index(['date', 'home_flag_team1', 'home_flag_team2', 'score1', 'score2',
       'season', 'team1', 'team2', 'win_flag_team1', 'win_flag_team2',
       'game_number_team1_x', 'game_number_team2_x', 'b2b_flag_team1',
       'b2b_flag_team2', 'travel_flag_team1', 'travel_flag_team2',
       'team1_game_id', 'team2_game_id', 'game_number_team1_y', 'poss_team1',
       'ortg_team1', 'ast%_team1', 'ts%_team1', 'efg%_team1', 'oreb_team1',
       'dreb_team1', 'blk_team1', 'stl_team1', 'pf_team1', 'to_team1',
       'fga_team1', 'fta_team1', '3fga_team1', 'fgm_team1', 'ftm_team1',
       '3fgm_team1', 'game_number_team2_y', 'poss_team2', 'ortg_team2',
       'ast%_team2', 'ts%_team2', 'efg%_team2', 'oreb_team2', 'dreb_team2',
       'blk_team2', 'stl_team2', 'pf_team2', 'to_team2', 'fga_team2',
       'fta_team2', '3fga_team2', 'fgm_team2', 'ftm_team2', '3fgm_team2'],
      dtype='object')

In [10]:
game_results.head()

,date,home_flag_team1,home_flag_team2,score1,score2,season,team1,team2,win_flag_team1,win_flag_team2,...,blk_team2,stl_team2,pf_team2,to_team2,fga_team2,fta_team2,3fga_team2,fgm_team2,ftm_team2,3fgm_team2
0,1999-02-05,1,0,100,83,1999,ATLANTA HAWKS,CLEVELAND CAVALIERS,1,0,...,0,11,31,17,78,30,3,32,18,1
1,1999-02-06,1,0,111,106,1999,ATLANTA HAWKS,NEW JERSEY NETS,1,0,...,3,7,38,9,91,38,9,38,28,2
2,1999-02-07,0,1,69,79,1999,ATLANTA HAWKS,NEW JERSEY NETS,0,1,...,10,6,27,22,65,33,8,26,25,2
3,1999-02-09,0,1,87,71,1999,ATLANTA HAWKS,CHICAGO BULLS,1,0,...,6,8,28,11,87,14,13,32,5,2
4,1999-02-11,1,0,100,88,1999,ATLANTA HAWKS,CHARLOTTE HORNETS,1,0,...,6,5,24,8,86,16,14,37,12,2


In [11]:
# create more normalized fields as features :
# teams def rating
# reb%, blk%, stl%, tov%, stop%, rolling win %, 

game_results['dreb%_team1'] = (game_results['dreb_team1']/(game_results['dreb_team1']+game_results['oreb_team2']))
game_results['oreb%_team1'] = (game_results['oreb_team1']/(game_results['oreb_team1']+game_results['dreb_team2']))
game_results['stl%_team1'] = (game_results['stl_team1']/game_results['poss_team2'])
game_results['blk%_team1']= (game_results['blk_team1']/game_results['poss_team2'])
game_results['tov%_team1'] = (game_results['to_team1']/game_results['poss_team1'])
game_results['fg%_team1'] =  (game_results['fgm_team1']/game_results['fga_team1'])
game_results['3fg%_team1'] = (game_results['3fgm_team1']/game_results['3fga_team1'])
game_results['ft%_team1'] =  (game_results['ftm_team1']/game_results['fta_team1'])
game_results['dreb%_team2'] = (game_results['dreb_team2']/(game_results['dreb_team2']+game_results['oreb_team1']))
game_results['oreb%_team2'] = (game_results['oreb_team2']/(game_results['oreb_team2']+game_results['dreb_team1']))
game_results['stl%_team2'] = (game_results['stl_team2']/game_results['poss_team1'])
game_results['blk%_team2']= (game_results['blk_team2']/game_results['poss_team1'])
game_results['tov%_team2'] = (game_results['to_team2']/game_results['poss_team2'])
game_results['3fg%_team2'] =  (game_results['3fgm_team2']/game_results['3fga_team2'])
game_results['fg%_team2'] =  (game_results['fgm_team2']/game_results['fga_team2'])
game_results['ft%_team2'] =  (game_results['ftm_team2']/game_results['fta_team2'])

game_results_advanced = game_results[['date','team1','team2','season','game_number_team1_x','game_number_team2_x','team1_game_id','team2_game_id','home_flag_team1', 'home_flag_team2',
                                   'win_flag_team1','win_flag_team2','ortg_team1','ast%_team1','ts%_team1','efg%_team1','poss_team1','b2b_flag_team1', 'travel_flag_team1',
                                   'dreb%_team1','oreb%_team1','stl%_team1','blk%_team1','tov%_team1','fg%_team1','3fg%_team1','ft%_team1',
                                   'ortg_team2','ast%_team2','ts%_team2','efg%_team2','poss_team2','b2b_flag_team2', 'travel_flag_team2',
                                   'dreb%_team2','oreb%_team2','stl%_team2','blk%_team2','tov%_team2','fg%_team2','3fg%_team2','ft%_team2']]
game_results_advanced.head()


,date,team1,team2,season,game_number_team1_x,game_number_team2_x,team1_game_id,team2_game_id,home_flag_team1,home_flag_team2,...,b2b_flag_team2,travel_flag_team2,dreb%_team2,oreb%_team2,stl%_team2,blk%_team2,tov%_team2,fg%_team2,3fg%_team2,ft%_team2
0,1999-02-05,ATLANTA HAWKS,CLEVELAND CAVALIERS,1999,1,1,ATLANTA HAWKS-1999-1,CLEVELAND CAVALIERS-1999-1,1,0,...,0,1,0.714286,0.317073,0.120614,0.000000,0.180851,0.410256,0.333333,0.600000
1,1999-02-06,ATLANTA HAWKS,NEW JERSEY NETS,1999,2,1,ATLANTA HAWKS-1999-2,NEW JERSEY NETS-1999-1,1,0,...,0,1,0.625000,0.361702,0.075269,0.032258,0.091650,0.417582,0.222222,0.736842
2,1999-02-07,ATLANTA HAWKS,NEW JERSEY NETS,1999,3,2,ATLANTA HAWKS-1999-3,NEW JERSEY NETS-1999-2,0,1,...,1,1,0.666667,0.411765,0.065646,0.109409,0.255220,0.400000,0.250000,0.757576
3,1999-02-09,ATLANTA HAWKS,CHICAGO BULLS,1999,4,3,ATLANTA HAWKS-1999-4,CHICAGO BULLS-1999-3,0,1,...,0,1,0.636364,0.340000,0.093458,0.070093,0.127021,0.367816,0.153846,0.357143
4,1999-02-11,ATLANTA HAWKS,CHARLOTTE HORNETS,1999,5,5,ATLANTA HAWKS-1999-5,CHARLOTTE HORNETS-1999-5,1,0,...,1,1,0.700000,0.116279,0.055679,0.066815,0.083857,0.430233,0.142857,0.750000


In [12]:
# create winning percentage metrics: season, b2b/break, travel/home-home, home/away, rolling 8 games
game_results_advanced = game_results_advanced.sort_values('date')

# season
game_results_advanced['wins_team1'] = game_results_advanced.groupby(['team1','season'])['win_flag_team1'].cumsum()
game_results_advanced['win%_team1'] = np.where(game_results_advanced['win_flag_team1']==1,(game_results_advanced['wins_team1']-1)/(game_results_advanced['game_number_team1_x']-1),game_results_advanced['wins_team1']/(game_results_advanced['game_number_team1_x']-1))

# home/away win percentage
game_results_advanced['total_homegames_team1'] = game_results_advanced.groupby(['team1','season'])['home_flag_team1'].cumsum()
game_results_advanced['total_homewins_team1'] = game_results_advanced.groupby(['team1','season','home_flag_team1'])['win_flag_team1'].cumsum()
game_results_advanced['away_flag_team1'] = np.where(game_results_advanced['home_flag_team1']==1,0,1)
game_results_advanced['total_awaygames_team1'] = game_results_advanced.groupby(['team1','season'])['away_flag_team1'].cumsum()
game_results_advanced['total_awaywins_team1'] = game_results_advanced.groupby(['team1','season','away_flag_team1'])['win_flag_team1'].cumsum()
game_results_advanced.loc[(game_results_advanced['home_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 1)
                          ,'homeaway_win%_team1'] = (game_results_advanced['total_homewins_team1']-1)/(game_results_advanced['total_homegames_team1']-1)
game_results_advanced.loc[(game_results_advanced['home_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 0)
                          ,'homeaway_win%_team1'] = (game_results_advanced['total_homewins_team1'])/(game_results_advanced['total_homegames_team1']-1)  
game_results_advanced.loc[(game_results_advanced['away_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 1)
                          ,'homeaway_win%_team1'] = (game_results_advanced['total_awaywins_team1']-1)/(game_results_advanced['total_awaygames_team1']-1)
game_results_advanced.loc[(game_results_advanced['away_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 0)
                          ,'homeaway_win%_team1'] = (game_results_advanced['total_awaywins_team1'])/(game_results_advanced['total_awaygames_team1']-1)  

# b2b/break win percentage
game_results_advanced['total_b2bgames_team1'] = game_results_advanced.groupby(['team1','season'])['b2b_flag_team1'].cumsum()
game_results_advanced['total_b2bwins_team1'] = game_results_advanced.groupby(['team1','season','b2b_flag_team1'])['win_flag_team1'].cumsum()
game_results_advanced['break_flag_team1'] = np.where(game_results_advanced['b2b_flag_team1']==1,0,1)
game_results_advanced['total_breakgames_team1'] = game_results_advanced.groupby(['team1','season'])['break_flag_team1'].cumsum()
game_results_advanced['total_breakwins_team1'] = game_results_advanced.groupby(['team1','season','break_flag_team1'])['win_flag_team1'].cumsum()
game_results_advanced.loc[(game_results_advanced['b2b_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 1)
                          ,'b2bbreak_win%_team1'] = (game_results_advanced['total_b2bwins_team1']-1)/(game_results_advanced['total_b2bgames_team1']-1)
game_results_advanced.loc[(game_results_advanced['b2b_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 0)
                          ,'b2bbreak_win%_team1'] = (game_results_advanced['total_b2bwins_team1'])/(game_results_advanced['total_b2bgames_team1']-1)
game_results_advanced.loc[(game_results_advanced['break_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 1)
                          ,'b2bbreak_win%_team1'] = (game_results_advanced['total_breakwins_team1']-1)/(game_results_advanced['total_breakgames_team1']-1)
game_results_advanced.loc[(game_results_advanced['break_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 0)
                          ,'b2bbreak_win%_team1'] = (game_results_advanced['total_breakwins_team1'])/(game_results_advanced['total_breakgames_team1']-1)

# travel/non travel win percentage
game_results_advanced['total_travelgames_team1'] = game_results_advanced.groupby(['team1','season'])['travel_flag_team1'].cumsum()
game_results_advanced['total_travelwins_team1'] = game_results_advanced.groupby(['team1','season','travel_flag_team1'])['win_flag_team1'].cumsum()
game_results_advanced['notravel_flag_team1'] = np.where(game_results_advanced['travel_flag_team1']==1,0,1)
game_results_advanced['total_notravelgames_team1'] = game_results_advanced.groupby(['team1','season'])['notravel_flag_team1'].cumsum()
game_results_advanced['total_notravelwins_team1'] = game_results_advanced.groupby(['team1','season','notravel_flag_team1'])['win_flag_team1'].cumsum()
game_results_advanced.loc[(game_results_advanced['travel_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 1)
                          ,'travel_win%_team1'] = (game_results_advanced['total_travelwins_team1']-1)/(game_results_advanced['total_travelgames_team1']-1)
game_results_advanced.loc[(game_results_advanced['travel_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 0)
                          ,'travel_win%_team1'] = (game_results_advanced['total_travelwins_team1'])/(game_results_advanced['total_travelgames_team1']-1)
game_results_advanced.loc[(game_results_advanced['notravel_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 1)
                          ,'travel_win%_team1'] = (game_results_advanced['total_notravelwins_team1']-1)/(game_results_advanced['total_notravelgames_team1']-1)
game_results_advanced.loc[(game_results_advanced['notravel_flag_team1'] == 1) & (game_results_advanced['win_flag_team1'] == 0)
                          ,'travel_win%_team1'] = (game_results_advanced['total_notravelwins_team1'])/(game_results_advanced['total_notravelgames_team1']-1)


# Rinse and repeat for team2
# season
game_results_advanced['wins_team2'] = game_results_advanced.groupby(['team2','season'])['win_flag_team2'].cumsum()
game_results_advanced['win%_team2'] = np.where(game_results_advanced['win_flag_team2']==1,
                                               (game_results_advanced['wins_team2']-1)/(game_results_advanced['game_number_team2_x']-1),
                                               game_results_advanced['wins_team2']/(game_results_advanced['game_number_team2_x']-1))

# home/away win percentage
game_results_advanced['total_homegames_team2'] = game_results_advanced.groupby(['team2','season'])['home_flag_team2'].cumsum()
game_results_advanced['total_homewins_team2'] = game_results_advanced.groupby(['team2','season','home_flag_team2'])['win_flag_team2'].cumsum()
game_results_advanced['away_flag_team2'] = np.where(game_results_advanced['home_flag_team2']==1,0,1)
game_results_advanced['total_awaygames_team2'] = game_results_advanced.groupby(['team2','season'])['away_flag_team2'].cumsum()
game_results_advanced['total_awaywins_team2'] = game_results_advanced.groupby(['team2','season','away_flag_team2'])['win_flag_team2'].cumsum()
game_results_advanced.loc[(game_results_advanced['home_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 1)
                          ,'homeaway_win%_team2'] = (game_results_advanced['total_homewins_team2']-1)/(game_results_advanced['total_homegames_team2']-1)
game_results_advanced.loc[(game_results_advanced['home_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 0)
                          ,'homeaway_win%_team2'] = (game_results_advanced['total_homewins_team2'])/(game_results_advanced['total_homegames_team2']-1)  
game_results_advanced.loc[(game_results_advanced['away_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 1)
                          ,'homeaway_win%_team2'] = (game_results_advanced['total_awaywins_team2']-1)/(game_results_advanced['total_awaygames_team2']-1)
game_results_advanced.loc[(game_results_advanced['away_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 0)
                          ,'homeaway_win%_team2'] = (game_results_advanced['total_awaywins_team2'])/(game_results_advanced['total_awaygames_team2']-1)  

# b2b/break win percentage
game_results_advanced['total_b2bgames_team2'] = game_results_advanced.groupby(['team2','season'])['b2b_flag_team2'].cumsum()
game_results_advanced['total_b2bwins_team2'] = game_results_advanced.groupby(['team2','season','b2b_flag_team2'])['win_flag_team2'].cumsum()
game_results_advanced['break_flag_team2'] = np.where(game_results_advanced['b2b_flag_team2']==1,0,1)
game_results_advanced['total_breakgames_team2'] = game_results_advanced.groupby(['team2','season'])['break_flag_team2'].cumsum()
game_results_advanced['total_breakwins_team2'] = game_results_advanced.groupby(['team2','season','break_flag_team2'])['win_flag_team2'].cumsum()
game_results_advanced.loc[(game_results_advanced['b2b_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 1)
                          ,'b2bbreak_win%_team2'] = (game_results_advanced['total_b2bwins_team2']-1)/(game_results_advanced['total_b2bgames_team2']-1)
game_results_advanced.loc[(game_results_advanced['b2b_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 0)
                          ,'b2bbreak_win%_team2'] = (game_results_advanced['total_b2bwins_team2'])/(game_results_advanced['total_b2bgames_team2']-1)
game_results_advanced.loc[(game_results_advanced['break_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 1)
                          ,'b2bbreak_win%_team2'] = (game_results_advanced['total_breakwins_team2']-1)/(game_results_advanced['total_breakgames_team2']-1)
game_results_advanced.loc[(game_results_advanced['break_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 0)
                          ,'b2bbreak_win%_team2'] = (game_results_advanced['total_breakwins_team2'])/(game_results_advanced['total_breakgames_team2']-1)

# travel/non travel win percentage
game_results_advanced['total_travelgames_team2'] = game_results_advanced.groupby(['team2','season'])['travel_flag_team2'].cumsum()
game_results_advanced['total_travelwins_team2'] = game_results_advanced.groupby(['team2','season','travel_flag_team2'])['win_flag_team2'].cumsum()
game_results_advanced['notravel_flag_team2'] = np.where(game_results_advanced['travel_flag_team2']==1,0,1)
game_results_advanced['total_notravelgames_team2'] = game_results_advanced.groupby(['team2','season'])['notravel_flag_team2'].cumsum()
game_results_advanced['total_notravelwins_team2'] = game_results_advanced.groupby(['team2','season','notravel_flag_team2'])['win_flag_team2'].cumsum()
game_results_advanced.loc[(game_results_advanced['travel_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 1)
                          ,'travel_win%_team2'] = (game_results_advanced['total_travelwins_team2']-1)/(game_results_advanced['total_travelgames_team2']-1)
game_results_advanced.loc[(game_results_advanced['travel_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 0)
                          ,'travel_win%_team2'] = (game_results_advanced['total_travelwins_team2'])/(game_results_advanced['total_travelgames_team2']-1)
game_results_advanced.loc[(game_results_advanced['notravel_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 1)
                          ,'travel_win%_team2'] = (game_results_advanced['total_notravelwins_team2']-1)/(game_results_advanced['total_notravelgames_team2']-1)
game_results_advanced.loc[(game_results_advanced['notravel_flag_team2'] == 1) & (game_results_advanced['win_flag_team2'] == 0)
                          ,'travel_win%_team2'] = (game_results_advanced['total_notravelwins_team2'])/(game_results_advanced['total_notravelgames_team2']-1)


In [13]:
# replace all nulls as 0
descriptives = game_results_advanced.describe()
descriptives = descriptives.transpose()
game_results_advanced.to_csv('game_results_advanced.csv')
game_results_advanced = game_results_advanced.fillna(0)
game_results_advanced = game_results_advanced[(game_results_advanced['win%_team1']<=1) & (game_results_advanced['win%_team2']<=1)]
print(len(game_results_advanced))

51643


In [14]:
game_results_records = game_results_advanced[['date','team1','season','wins_team1','game_number_team1_x','wins_team2','game_number_team2_x','team2','team1_game_id','team2_game_id','ortg_team1','ast%_team1','ts%_team1','efg%_team1','poss_team1',
                                   'dreb%_team1','oreb%_team1','stl%_team1','blk%_team1','tov%_team1','fg%_team1','3fg%_team1','ft%_team1',
                                    'win%_team1','homeaway_win%_team1','b2bbreak_win%_team1','travel_win%_team1',
                                   'ortg_team2','ast%_team2','ts%_team2','efg%_team2','poss_team2',
                                   'dreb%_team2','oreb%_team2','stl%_team2','blk%_team2','tov%_team2','fg%_team2','3fg%_team2','ft%_team2',
                                    'win%_team2','homeaway_win%_team2','b2bbreak_win%_team2','travel_win%_team2','win_flag_team1']]

In [15]:
game_results_records_atl = game_results_records[game_results_records.team1=='ATLANTA HAWKS']
game_results_records.to_csv('game_results_records.csv')
game_results_records_atl.to_csv('game_results_records_atl.csv')
game_results_records.head()

,date,team1,season,wins_team1,game_number_team1_x,wins_team2,game_number_team2_x,team2,team1_game_id,team2_game_id,...,blk%_team2,tov%_team2,fg%_team2,3fg%_team2,ft%_team2,win%_team2,homeaway_win%_team2,b2bbreak_win%_team2,travel_win%_team2,win_flag_team1
0,1999-02-05,ATLANTA HAWKS,1999,1,1,0,1,CLEVELAND CAVALIERS,ATLANTA HAWKS-1999-1,CLEVELAND CAVALIERS-1999-1,...,0.000000,0.180851,0.410256,0.333333,0.600000,0.0,0.0,0.0,0.0,1
4939,1999-02-05,CHARLOTTE HORNETS,1999,0,1,1,1,PHILADELPHIA 76ERS,CHARLOTTE HORNETS-1999-1,PHILADELPHIA 76ERS-1999-1,...,0.115473,0.135747,0.380952,0.250000,0.687500,0.0,0.0,0.0,0.0,0
5710,1999-02-05,CHICAGO BULLS,1999,0,1,1,1,UTAH JAZZ,CHICAGO BULLS-1999-1,UTAH JAZZ-1999-1,...,0.098253,0.213964,0.576271,0.500000,0.785714,0.0,0.0,0.0,0.0,0
47960,1999-02-05,UTAH JAZZ,1999,1,1,0,1,CHICAGO BULLS,UTAH JAZZ-1999-1,CHICAGO BULLS-1999-1,...,0.033784,0.131004,0.435294,0.461538,0.666667,0.0,0.0,0.0,0.0,1
7451,1999-02-05,CLEVELAND CAVALIERS,1999,0,1,1,1,ATLANTA HAWKS,CLEVELAND CAVALIERS-1999-1,ATLANTA HAWKS-1999-1,...,0.085106,0.142544,0.464789,0.200000,0.744186,0.0,0.0,0.0,0.0,0


In [ ]:
## notes 07-03-20 

# rolling 10 games winning pct
# consider creating a class/function to clean code
# consider looking at feature space and correlation matrix (after looking at baseline model results)

# goals for 7-9-20

# wrap up wrangling
#     - qa advanced metrics (looks good here)
#     - include rolling 8 game averages (for advanced and winning pct metrics)
# push code to git
# lecture work
# start modelling (log, random forest, nn, gradientboost, tune all above, ensemble)
# fix mini projects 


In [16]:
# get rolling results
# join all advanced metrics to box_score_table

box_score_advanced = all_boxscore_results_df.merge(right = game_results_records, how='inner',  left_on='game_id', right_on='team1_game_id')
box_score_advanced = box_score_advanced[['season_x','team1', 'team1_game_id','wins_team1','game_number_team1_x', 
                                         'ortg_team1', 'ast%_team1','ts%_team1', 'efg%_team1', 'poss_team1',
                                         'dreb%_team1', 'oreb%_team1','stl%_team1', 'blk%_team1', 'tov%_team1', 
                                         'fg%_team1', '3fg%_team1','ft%_team1', 'win%_team1', 'homeaway_win%_team1',
                                         'b2bbreak_win%_team1','travel_win%_team1',
                                         'team2','team2_game_id','wins_team2','game_number_team2_x',
                                         'ortg_team2', 'ast%_team2', 'ts%_team2','efg%_team2','poss_team2',
                                         'dreb%_team2', 'oreb%_team2', 'stl%_team2','blk%_team2', 'tov%_team2',
                                         'fg%_team2', '3fg%_team2', 'ft%_team2','win%_team2', 'homeaway_win%_team2',
                                         'b2bbreak_win%_team2','travel_win%_team2', 'win_flag_team1']]

box_score_advanced = box_score_advanced.rename(columns={"season_x":"season",
                                                          "team1":"team1",
                                                          "team1_game_id":"game_id_team1",
                                                          "wins_team1":"wins_team1",
                                                          "game_number_team1_x":"game_number_team1",
                                                          "ortg_team1":"ortg_team1",
                                                          "ast%_team1":"ast%_team1",
                                                          "ts%_team1":"ts%_team1",
                                                          "efg%_team1":"efg%_team1",
                                                          "poss_team1":"poss_team1",
                                                          "dreb%_team1":"dreb%_team1",
                                                          "oreb%_team1":"oreb%_team1",
                                                           "stl%_team1": "stl%_team1",
                                                           "blk%_team1": "blk%_team1",
                                                           "tov%_team1": "tov%_team1",
                                                         "win%_team1":"win%_team1",
                                                          "homeaway_win%_team1":"homeaway_win%_team1",
                                                          "b2bbreak_win%_team1":"b2bbreak_win%_team1",
                                                          "travel_win%_team1":"travel_win%_team1",
                                                          "team2":"team1_opp",
                                                          "team2_game_id":"game_id_team1_opp",
                                                          "wins_team2":"wins_team1_opp",
                                                          "game_number_team2_x":"game_number_team1_opp",
                                                          "ortg_team2":"ortg_team1_opp",
                                                          "ast%_team2":"ast%_team1_opp",
                                                          "ts%_team2":"ts%_team1_opp",
                                                          "efg%_team2":"efg%_team1_opp",
                                                          "poss_team2":"poss_team1_opp",
                                                          "dreb%_team2":"dreb%_team1_opp",
                                                          "oreb%_team2":"oreb%_team1_opp",
                                                           "stl%_team2": "stl%_team1_opp",
                                                           "blk%_team2": "blk%_team1_opp",
                                                           "tov%_team2": "tov%_team1_opp",
                                                         "win%_team2":"win%_team1_opp",
                                                          "homeaway_win%_team2":"homeaway_win%_team1_opp",
                                                          "b2bbreak_win%_team2":"b2bbreak_win%_team1_opp",
                                                          "travel_win%_team2":"travel_win%_team1_opp",
                                                          "ft%_team2":"ft%_team1_opp"
                                                          })
print(len(box_score_advanced))
box_score_advanced.to_csv('advanced box score.csv')

51643


In [17]:
box_score_advanced.head()

,season,team1,game_id_team1,wins_team1,game_number_team1,ortg_team1,ast%_team1,ts%_team1,efg%_team1,poss_team1,...,blk%_team1_opp,tov%_team1_opp,fg%_team2,3fg%_team2,ft%_team1_opp,win%_team1_opp,homeaway_win%_team1_opp,b2bbreak_win%_team1_opp,travel_win%_team1_opp,win_flag_team1
0,1999,ATLANTA HAWKS,ATLANTA HAWKS-1999-1,1,1,109.649123,0.484848,0.556050,0.478873,91.2,...,0.000000,0.180851,0.410256,0.333333,0.600000,0.00,0.0,0.0,0.0,1
1,1999,ATLANTA HAWKS,ATLANTA HAWKS-1999-2,2,2,119.354839,0.500000,0.600649,0.477273,93.0,...,0.032258,0.091650,0.417582,0.222222,0.736842,0.00,0.0,0.0,0.0,1
2,1999,ATLANTA HAWKS,ATLANTA HAWKS-1999-3,2,3,75.492341,0.391304,0.385733,0.307692,91.4,...,0.109409,0.255220,0.400000,0.250000,0.757576,0.00,0.0,0.0,0.0,0
3,1999,ATLANTA HAWKS,ATLANTA HAWKS-1999-4,3,4,101.635514,0.484848,0.525616,0.492857,85.6,...,0.070093,0.127021,0.367816,0.153846,0.357143,0.50,0.0,0.5,0.5,1
4,1999,ATLANTA HAWKS,ATLANTA HAWKS-1999-5,4,5,111.358575,0.615385,0.537172,0.500000,89.8,...,0.066815,0.083857,0.430233,0.142857,0.750000,0.25,0.0,0.0,0.0,1


In [18]:
box_score_advanced.columns

Index(['season', 'team1', 'game_id_team1', 'wins_team1', 'game_number_team1',
       'ortg_team1', 'ast%_team1', 'ts%_team1', 'efg%_team1', 'poss_team1',
       'dreb%_team1', 'oreb%_team1', 'stl%_team1', 'blk%_team1', 'tov%_team1',
       'fg%_team1', '3fg%_team1', 'ft%_team1', 'win%_team1',
       'homeaway_win%_team1', 'b2bbreak_win%_team1', 'travel_win%_team1',
       'team1_opp', 'game_id_team1_opp', 'wins_team1_opp',
       'game_number_team1_opp', 'ortg_team1_opp', 'ast%_team1_opp',
       'ts%_team1_opp', 'efg%_team1_opp', 'poss_team1_opp', 'dreb%_team1_opp',
       'oreb%_team1_opp', 'stl%_team1_opp', 'blk%_team1_opp', 'tov%_team1_opp',
       'fg%_team2', '3fg%_team2', 'ft%_team1_opp', 'win%_team1_opp',
       'homeaway_win%_team1_opp', 'b2bbreak_win%_team1_opp',
       'travel_win%_team1_opp', 'win_flag_team1'],
      dtype='object')

In [19]:
team_list = box_score_advanced.team1.unique()
season_list = box_score_advanced.season.unique()
box_score_advanced_roll8_list = []
box_score_advanced_roll8 = pd.DataFrame(columns = ['team1','season','game_number_team1','ortg_team1','ast%_team1', 'ts%_team1',
                                                   'efg%_team1', 'poss_team1', 'dreb%_team1', 'oreb%_team1', 'stl%_team1',
                                                   'blk%_team1', 'tov%_team1', 'ft%_team1','win%_roll8_team1',
                                                   'ortg_team1_opp', 'ast%_team1_opp', 'ts%_team1_opp', 'efg%_team1_opp', 
                                                   'poss_team1_opp', 'dreb%_team1_opp', 'oreb%_team1_opp', 'stl%_team1_opp', 
                                                   'blk%_team1_opp', 'tov%_team1_opp','ft%_team1_opp']) 
for team in team_list:
    for season in season_list:
        team_advanced_stats = box_score_advanced[(box_score_advanced['team1'] == team) & (box_score_advanced['season'] == season)]
        for game_number in range(2,len(team_advanced_stats)+1):
            if game_number <= 9:
                rolling_8_df = team_advanced_stats[:game_number-1]
            else:
                rolling_8_df = team_advanced_stats[game_number-9:game_number-1]
            roll8_adv_met_list = [
            team,
            season,
            game_number,
            rolling_8_df['ortg_team1'].mean(),
            rolling_8_df['ast%_team1'].mean(),
            rolling_8_df['ts%_team1'].mean(),
            rolling_8_df['efg%_team1'].mean(),
            rolling_8_df['poss_team1'].mean(),
            rolling_8_df['dreb%_team1'].mean(),
            rolling_8_df['oreb%_team1'].mean(),
            rolling_8_df['stl%_team1'].mean(),
            rolling_8_df['blk%_team1'].mean(),
            rolling_8_df['tov%_team1'].mean(),
            rolling_8_df['ft%_team1'].mean(),
            rolling_8_df['win_flag_team1'].sum()/len(rolling_8_df),
            rolling_8_df['ortg_team1_opp'].mean(),
            rolling_8_df['ast%_team1_opp'].mean(),
            rolling_8_df['ts%_team1_opp'].mean(),
            rolling_8_df['efg%_team1_opp'].mean(),
            rolling_8_df['poss_team1_opp'].mean(),
            rolling_8_df['dreb%_team1_opp'].mean(),
            rolling_8_df['oreb%_team1_opp'].mean(),
            rolling_8_df['stl%_team1_opp'].mean(),
            rolling_8_df['blk%_team1_opp'].mean(),
            rolling_8_df['tov%_team1_opp'].mean(),
            rolling_8_df['ft%_team1_opp'].mean()
            ]
            box_score_advanced_roll8_list.append(roll8_adv_met_list)
            Var = pd.DataFrame(box_score_advanced_roll8_list)
            Var.columns = ['team1','season','game_number_team1','ortg_team1','ast%_team1', 'ts%_team1',
                           'efg%_team1', 'poss_team1', 'dreb%_team1', 'oreb%_team1', 'stl%_team1',
                           'blk%_team1', 'tov%_team1', 'ft%_team1','win%_roll8_team1',
                           'ortg_team1_opp', 'ast%_team1_opp', 'ts%_team1_opp', 'efg%_team1_opp', 
                           'poss_team1_opp', 'dreb%_team1_opp', 'oreb%_team1_opp', 'stl%_team1_opp', 
                           'blk%_team1_opp', 'tov%_team1_opp','ft%_team1_opp']
        print("{}'s {} season rolling avg metrics is completed. Dataset size: {}".format(team,season,len(Var)))
box_score_advanced_roll8 = box_score_advanced_roll8.append(Var)

ATLANTA HAWKS's 1999 season rolling avg metrics is completed. Dataset size: 58
ATLANTA HAWKS's 2000 season rolling avg metrics is completed. Dataset size: 139
ATLANTA HAWKS's 2001 season rolling avg metrics is completed. Dataset size: 220
ATLANTA HAWKS's 2002 season rolling avg metrics is completed. Dataset size: 301
ATLANTA HAWKS's 2003 season rolling avg metrics is completed. Dataset size: 382
ATLANTA HAWKS's 2004 season rolling avg metrics is completed. Dataset size: 463
ATLANTA HAWKS's 2005 season rolling avg metrics is completed. Dataset size: 544
ATLANTA HAWKS's 2006 season rolling avg metrics is completed. Dataset size: 623
ATLANTA HAWKS's 2007 season rolling avg metrics is completed. Dataset size: 702
ATLANTA HAWKS's 2008 season rolling avg metrics is completed. Dataset size: 783
ATLANTA HAWKS's 2009 season rolling avg metrics is completed. Dataset size: 864
ATLANTA HAWKS's 2010 season rolling avg metrics is completed. Dataset size: 945
ATLANTA HAWKS's 2011 season rolling avg m

CHARLOTTE HORNETS's 2015 season rolling avg metrics is completed. Dataset size: 5249
CHARLOTTE HORNETS's 2016 season rolling avg metrics is completed. Dataset size: 5330
CHARLOTTE HORNETS's 2017 season rolling avg metrics is completed. Dataset size: 5411
CHARLOTTE HORNETS's 2018 season rolling avg metrics is completed. Dataset size: 5492
CHARLOTTE HORNETS's 2019 season rolling avg metrics is completed. Dataset size: 5573
CHARLOTTE HORNETS's 2020 season rolling avg metrics is completed. Dataset size: 5637
CHICAGO BULLS's 1999 season rolling avg metrics is completed. Dataset size: 5686
CHICAGO BULLS's 2000 season rolling avg metrics is completed. Dataset size: 5767
CHICAGO BULLS's 2001 season rolling avg metrics is completed. Dataset size: 5848
CHICAGO BULLS's 2002 season rolling avg metrics is completed. Dataset size: 5929
CHICAGO BULLS's 2003 season rolling avg metrics is completed. Dataset size: 6010
CHICAGO BULLS's 2004 season rolling avg metrics is completed. Dataset size: 6091
CHIC

DETROIT PISTONS's 2003 season rolling avg metrics is completed. Dataset size: 12878
DETROIT PISTONS's 2004 season rolling avg metrics is completed. Dataset size: 12959
DETROIT PISTONS's 2005 season rolling avg metrics is completed. Dataset size: 13040
DETROIT PISTONS's 2006 season rolling avg metrics is completed. Dataset size: 13119
DETROIT PISTONS's 2007 season rolling avg metrics is completed. Dataset size: 13198
DETROIT PISTONS's 2008 season rolling avg metrics is completed. Dataset size: 13279
DETROIT PISTONS's 2009 season rolling avg metrics is completed. Dataset size: 13360
DETROIT PISTONS's 2010 season rolling avg metrics is completed. Dataset size: 13441
DETROIT PISTONS's 2011 season rolling avg metrics is completed. Dataset size: 13522
DETROIT PISTONS's 2012 season rolling avg metrics is completed. Dataset size: 13587
DETROIT PISTONS's 2013 season rolling avg metrics is completed. Dataset size: 13668
DETROIT PISTONS's 2014 season rolling avg metrics is completed. Dataset size

LOS ANGELES CLIPPERS's 2011 season rolling avg metrics is completed. Dataset size: 20388
LOS ANGELES CLIPPERS's 2012 season rolling avg metrics is completed. Dataset size: 20464
LOS ANGELES CLIPPERS's 2013 season rolling avg metrics is completed. Dataset size: 20545
LOS ANGELES CLIPPERS's 2014 season rolling avg metrics is completed. Dataset size: 20626
LOS ANGELES CLIPPERS's 2015 season rolling avg metrics is completed. Dataset size: 20707
LOS ANGELES CLIPPERS's 2016 season rolling avg metrics is completed. Dataset size: 20788
LOS ANGELES CLIPPERS's 2017 season rolling avg metrics is completed. Dataset size: 20869
LOS ANGELES CLIPPERS's 2018 season rolling avg metrics is completed. Dataset size: 20950
LOS ANGELES CLIPPERS's 2019 season rolling avg metrics is completed. Dataset size: 21031
LOS ANGELES CLIPPERS's 2020 season rolling avg metrics is completed. Dataset size: 21094
LOS ANGELES LAKERS's 1999 season rolling avg metrics is completed. Dataset size: 21151
LOS ANGELES LAKERS's 20

MILWAUKEE BUCKS's 2020 season rolling avg metrics is completed. Dataset size: 27772
MINNESOTA TIMBERWOLVES's 1999 season rolling avg metrics is completed. Dataset size: 27825
MINNESOTA TIMBERWOLVES's 2000 season rolling avg metrics is completed. Dataset size: 27906
MINNESOTA TIMBERWOLVES's 2001 season rolling avg metrics is completed. Dataset size: 27987
MINNESOTA TIMBERWOLVES's 2002 season rolling avg metrics is completed. Dataset size: 28068
MINNESOTA TIMBERWOLVES's 2003 season rolling avg metrics is completed. Dataset size: 28149
MINNESOTA TIMBERWOLVES's 2004 season rolling avg metrics is completed. Dataset size: 28230
MINNESOTA TIMBERWOLVES's 2005 season rolling avg metrics is completed. Dataset size: 28311
MINNESOTA TIMBERWOLVES's 2006 season rolling avg metrics is completed. Dataset size: 28389
MINNESOTA TIMBERWOLVES's 2007 season rolling avg metrics is completed. Dataset size: 28466
MINNESOTA TIMBERWOLVES's 2008 season rolling avg metrics is completed. Dataset size: 28547
MINNES

NEW YORK KNICKS's 2004 season rolling avg metrics is completed. Dataset size: 32302
NEW YORK KNICKS's 2005 season rolling avg metrics is completed. Dataset size: 32383
NEW YORK KNICKS's 2006 season rolling avg metrics is completed. Dataset size: 32462
NEW YORK KNICKS's 2007 season rolling avg metrics is completed. Dataset size: 32541
NEW YORK KNICKS's 2008 season rolling avg metrics is completed. Dataset size: 32622
NEW YORK KNICKS's 2009 season rolling avg metrics is completed. Dataset size: 32703
NEW YORK KNICKS's 2010 season rolling avg metrics is completed. Dataset size: 32784
NEW YORK KNICKS's 2011 season rolling avg metrics is completed. Dataset size: 32865
NEW YORK KNICKS's 2012 season rolling avg metrics is completed. Dataset size: 32935
NEW YORK KNICKS's 2013 season rolling avg metrics is completed. Dataset size: 33015
NEW YORK KNICKS's 2014 season rolling avg metrics is completed. Dataset size: 33096
NEW YORK KNICKS's 2015 season rolling avg metrics is completed. Dataset size

PHOENIX SUNS's 2013 season rolling avg metrics is completed. Dataset size: 39138
PHOENIX SUNS's 2014 season rolling avg metrics is completed. Dataset size: 39219
PHOENIX SUNS's 2015 season rolling avg metrics is completed. Dataset size: 39300
PHOENIX SUNS's 2016 season rolling avg metrics is completed. Dataset size: 39381
PHOENIX SUNS's 2017 season rolling avg metrics is completed. Dataset size: 39462
PHOENIX SUNS's 2018 season rolling avg metrics is completed. Dataset size: 39543
PHOENIX SUNS's 2019 season rolling avg metrics is completed. Dataset size: 39624
PHOENIX SUNS's 2020 season rolling avg metrics is completed. Dataset size: 39688
PORTLAND TRAIL BLAZERS's 1999 season rolling avg metrics is completed. Dataset size: 39750
PORTLAND TRAIL BLAZERS's 2000 season rolling avg metrics is completed. Dataset size: 39831
PORTLAND TRAIL BLAZERS's 2001 season rolling avg metrics is completed. Dataset size: 39912
PORTLAND TRAIL BLAZERS's 2002 season rolling avg metrics is completed. Dataset 

TORONTO RAPTORS's 1999 season rolling avg metrics is completed. Dataset size: 45681
TORONTO RAPTORS's 2000 season rolling avg metrics is completed. Dataset size: 45762
TORONTO RAPTORS's 2001 season rolling avg metrics is completed. Dataset size: 45843
TORONTO RAPTORS's 2002 season rolling avg metrics is completed. Dataset size: 45924
TORONTO RAPTORS's 2003 season rolling avg metrics is completed. Dataset size: 46005
TORONTO RAPTORS's 2004 season rolling avg metrics is completed. Dataset size: 46086
TORONTO RAPTORS's 2005 season rolling avg metrics is completed. Dataset size: 46167
TORONTO RAPTORS's 2006 season rolling avg metrics is completed. Dataset size: 46246
TORONTO RAPTORS's 2007 season rolling avg metrics is completed. Dataset size: 46325
TORONTO RAPTORS's 2008 season rolling avg metrics is completed. Dataset size: 46406
TORONTO RAPTORS's 2009 season rolling avg metrics is completed. Dataset size: 46487
TORONTO RAPTORS's 2010 season rolling avg metrics is completed. Dataset size

In [20]:
box_score_advanced_roll8.head()

,team1,season,game_number_team1,ortg_team1,ast%_team1,ts%_team1,efg%_team1,poss_team1,dreb%_team1,oreb%_team1,...,ast%_team1_opp,ts%_team1_opp,efg%_team1_opp,poss_team1_opp,dreb%_team1_opp,oreb%_team1_opp,stl%_team1_opp,blk%_team1_opp,tov%_team1_opp,ft%_team1_opp
0,ATLANTA HAWKS,1999,2,109.649123,0.484848,0.556050,0.478873,91.200000,0.682927,0.285714,...,0.468750,0.455044,0.416667,94.00,0.714286,0.317073,0.120614,0.000000,0.180851,0.600000
1,ATLANTA HAWKS,1999,3,114.501981,0.492424,0.578350,0.478073,92.100000,0.660612,0.330357,...,0.418586,0.473530,0.422619,96.10,0.669643,0.339388,0.097941,0.016129,0.136250,0.668421
2,ATLANTA HAWKS,1999,4,101.498768,0.458718,0.514144,0.421279,91.866667,0.636487,0.331349,...,0.484185,0.481264,0.420208,92.80,0.668651,0.363513,0.087176,0.047222,0.175907,0.698139
3,ATLANTA HAWKS,1999,5,101.532954,0.465250,0.517012,0.439174,90.300000,0.642365,0.339421,...,0.503764,0.456214,0.409983,91.25,0.660579,0.357635,0.088747,0.052940,0.163685,0.612890
4,ATLANTA HAWKS,1999,6,103.498078,0.495277,0.521044,0.451339,90.200000,0.690636,0.331537,...,0.505714,0.459554,0.416359,92.08,0.668463,0.309364,0.082133,0.055715,0.147720,0.640312


In [21]:
box_score_advanced_roll8['season_game_team1_id']=box_score_advanced_roll8['team1'] + '-' + box_score_advanced_roll8['season'].apply(str) + '-' + box_score_advanced_roll8['game_number_team1'].apply(str)
box_score_advanced_roll8_team2 = box_score_advanced_roll8.copy(deep=True)
box_score_advanced_roll8_team2 = box_score_advanced_roll8_team2.rename(columns={
                                                                       'season_game_team1_id':'season_game_team2_id',
                                                                       'ortg_team1':'ortg_team2',
                                                                       'ast%_team1':'ast%_team2',
                                                                       'ts%_team1':'ts%_team2',
                                                                       'efg%_team1':'efg%_team2', 
                                                                       'poss_team1':'poss_team2',
                                                                       'dreb%_team1':'dreb%_team2',
                                                                       'oreb%_team1':'oreb%_team2',
                                                                       'stl%_team1':'stl%_team2',
                                                                       'blk%_team1':'blk%_team2',
                                                                       'tov%_team1':'tov%_team2',
                                                                       'ft%_team1':'ft%_team2',
                                                                       'win%_roll8_team1':'win%_roll8_team2',
                                                                       'ortg_team1_opp':'ortg_team2_opp',
                                                                       'ast%_team1_opp':'ast%_team2_opp',
                                                                       'ts%_team1_opp':'ts%_team2_opp',
                                                                       'efg%_team1_opp':'efg%_team2_opp', 
                                                                       'poss_team1_opp':'poss_team2_opp',
                                                                       'dreb%_team1_opp':'dreb%_team2_opp',
                                                                       'oreb%_team1_opp':'oreb%_team2_opp',
                                                                       'stl%_team1_opp':'stl%_team2_opp', 
                                                                       'blk%_team1_opp':'blk%_team2_opp', 
                                                                       'tov%_team1_opp':'tov%_team2_opp',
                                                                       'ft%_team1_opp':'ft%_team2_opp'
                                                                        })
box_score_advanced_roll8_team2.head()

,team1,season,game_number_team1,ortg_team2,ast%_team2,ts%_team2,efg%_team2,poss_team2,dreb%_team2,oreb%_team2,...,ts%_team2_opp,efg%_team2_opp,poss_team2_opp,dreb%_team2_opp,oreb%_team2_opp,stl%_team2_opp,blk%_team2_opp,tov%_team2_opp,ft%_team2_opp,season_game_team2_id
0,ATLANTA HAWKS,1999,2,109.649123,0.484848,0.556050,0.478873,91.200000,0.682927,0.285714,...,0.455044,0.416667,94.00,0.714286,0.317073,0.120614,0.000000,0.180851,0.600000,ATLANTA HAWKS-1999-2
1,ATLANTA HAWKS,1999,3,114.501981,0.492424,0.578350,0.478073,92.100000,0.660612,0.330357,...,0.473530,0.422619,96.10,0.669643,0.339388,0.097941,0.016129,0.136250,0.668421,ATLANTA HAWKS-1999-3
2,ATLANTA HAWKS,1999,4,101.498768,0.458718,0.514144,0.421279,91.866667,0.636487,0.331349,...,0.481264,0.420208,92.80,0.668651,0.363513,0.087176,0.047222,0.175907,0.698139,ATLANTA HAWKS-1999-4
3,ATLANTA HAWKS,1999,5,101.532954,0.465250,0.517012,0.439174,90.300000,0.642365,0.339421,...,0.456214,0.409983,91.25,0.660579,0.357635,0.088747,0.052940,0.163685,0.612890,ATLANTA HAWKS-1999-5
4,ATLANTA HAWKS,1999,6,103.498078,0.495277,0.521044,0.451339,90.200000,0.690636,0.331537,...,0.459554,0.416359,92.08,0.668463,0.309364,0.082133,0.055715,0.147720,0.640312,ATLANTA HAWKS-1999-6


In [22]:
win_pct_df = box_score_advanced[['game_id_team1', 
                                 'win%_team1','homeaway_win%_team1','b2bbreak_win%_team1', 'travel_win%_team1',
                                 'win%_team1_opp','homeaway_win%_team1_opp','b2bbreak_win%_team1_opp', 'travel_win%_team1_opp']]
win_pct_df_team2 = win_pct_df.copy(deep=True)
win_pct_df_team2 = win_pct_df_team2.rename(columns = {
                                            'game_id_team1':'game_id_team2', 
                                            'win%_team1':'win%_team2',
                                            'homeaway_win%_team1':'homeaway_win%_team2',
                                            'b2bbreak_win%_team1':'b2bbreak_win%_team2',
                                            'travel_win%_team1':'travel_win%_team2',
                                            'win%_team1_opp':'win%_team2_opp',
                                            'homeaway_win%_team1_opp':'homeaway_win%_team2_opp',
                                            'b2bbreak_win%_team1_opp':'b2bbreak_win%_team2_opp',
                                            'travel_win%_team1_opp':'travel_win%_team2_opp'
                                            })


In [23]:
game_live_results = pd.read_csv('all_season_results_final.csv')
game_live_results = game_live_results[['team1','season','team2','game_number_team1','game_number_team2','win_flag_team1']] 
game_live_results['season_game_team1_id'] = game_live_results['team1'] + '-' + game_live_results['season'].apply(str) + '-' + game_live_results['game_number_team1'].apply(str)
game_live_results['season_game_team2_id'] = game_live_results['team2'] + '-' + game_live_results['season'].apply(str) + '-' + game_live_results['game_number_team2'].apply(str)
final_df = game_live_results.merge(box_score_advanced_roll8, how = 'inner',  left_on='season_game_team1_id', right_on='season_game_team1_id')
final_df = final_df.merge(win_pct_df, how = 'inner',  left_on='season_game_team1_id', right_on='game_id_team1')
final_df = final_df.merge(box_score_advanced_roll8_team2, how = 'inner',  left_on='season_game_team2_id', right_on='season_game_team2_id')
final_df = final_df.merge(win_pct_df_team2, how = 'inner',  left_on='season_game_team2_id', right_on='game_id_team2')
final_df.head()

,team1_x,season_x,team2,game_number_team1_x,game_number_team2,win_flag_team1,season_game_team1_id,season_game_team2_id,team1_y,season_y,...,ft%_team2_opp,game_id_team2,win%_team2,homeaway_win%_team2,b2bbreak_win%_team2,travel_win%_team2,win%_team2_opp,homeaway_win%_team2_opp,b2bbreak_win%_team2_opp,travel_win%_team2_opp
0,ATLANTA HAWKS,1999,NEW JERSEY NETS,3,2,0,ATLANTA HAWKS-1999-3,NEW JERSEY NETS-1999-2,ATLANTA HAWKS,1999,...,0.800000,NEW JERSEY NETS-1999-2,0.00,0.000000,0.000000,0.0,1.000000,0.000000,1.00,0.000000
1,ATLANTA HAWKS,1999,CHICAGO BULLS,4,3,1,ATLANTA HAWKS-1999-4,CHICAGO BULLS-1999-3,ATLANTA HAWKS,1999,...,0.812857,CHICAGO BULLS-1999-3,0.50,0.000000,0.500000,0.5,0.666667,0.000000,1.00,0.000000
2,ATLANTA HAWKS,1999,CHARLOTTE HORNETS,5,5,1,ATLANTA HAWKS-1999-5,CHARLOTTE HORNETS-1999-5,ATLANTA HAWKS,1999,...,0.725852,CHARLOTTE HORNETS-1999-5,0.25,0.000000,0.000000,0.0,0.750000,1.000000,1.00,0.500000
3,ATLANTA HAWKS,1999,PHILADELPHIA 76ERS,6,5,0,ATLANTA HAWKS-1999-6,PHILADELPHIA 76ERS-1999-5,ATLANTA HAWKS,1999,...,0.593750,PHILADELPHIA 76ERS-1999-5,0.75,0.666667,0.666667,0.5,0.800000,0.500000,1.00,0.666667
4,ATLANTA HAWKS,1999,CHICAGO BULLS,7,6,1,ATLANTA HAWKS-1999-7,CHICAGO BULLS-1999-6,ATLANTA HAWKS,1999,...,0.709501,CHICAGO BULLS-1999-6,0.20,0.000000,0.200000,0.0,0.666667,0.333333,0.75,0.500000


In [24]:
final_df.to_csv('final_game_results.csv')